In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip
!rm glove.6B.zip
!tar -xzf /content/drive/MyDrive/NLP_Training/yelp_review_full_csv.tar.gz

--2024-04-21 14:31:51--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-04-21 14:31:51--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-04-21 14:31:51--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [2]:
!mkdir ./data
%cd data
!mkdir corpus
%cd ..

/content/data
/content


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Dữ liệu văn bản
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?'
]
vocabulary = ['the', 'first', 'document', 'is', 'second', 'third', 'and', 'one', 'this']
# Khởi tạo TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=vocabulary)

# Fit và transform dữ liệu
X = vectorizer.fit_transform(corpus)

# In ma trận TF-IDF
vocabulary = vectorizer.vocabulary_

# In danh sách các từ (features)
word_embeddings = {}
for word, index in vocabulary.items():
    word_embeddings[word] = X[:, index].toarray().squeeze()

print(word_embeddings)

{'the': array([0.38408524, 0.28108867, 0.26710379, 0.38408524]), 'first': array([0.58028582, 0.        , 0.        , 0.58028582]), 'document': array([0.46979139, 0.6876236 , 0.        , 0.46979139]), 'is': array([0.38408524, 0.28108867, 0.26710379, 0.38408524]), 'second': array([0.        , 0.53864762, 0.        , 0.        ]), 'third': array([0.        , 0.        , 0.51184851, 0.        ]), 'and': array([0.        , 0.        , 0.51184851, 0.        ]), 'one': array([0.        , 0.        , 0.51184851, 0.        ]), 'this': array([0.38408524, 0.28108867, 0.26710379, 0.38408524])}


In [3]:
import os
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
# from utils import load_corpus
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [24]:
def read_data(root_path):
    train_csv_path = os.path.join(root_path , "train.csv")
    test_csv_path = os.path.join(root_path , 'test.csv')
    dt_train = pd.read_csv(train_csv_path ,nrows=500).values
    dt_test = pd.read_csv(test_csv_path).values
    text_train = []
    label_train = []
    for x in dt_train:
        text_train.append(x[1].strip())
        label_train.append(str(x[0]))
    text_test = []
    label_test = []
    for x in dt_test:
        text_test.append(x[1].strip())
        label_test.append(str(x[0]))
    return text_train , label_train , text_test , label_test

def prepare_data():
    dataset_name = 'My_dataset'
    sentences , labels , st_test , st_test = read_data('/content/yelp_review_full_csv')
    train_or_test_list = ['train', 'test']
    meta_data_list = []
    for i in range(len(sentences)):
        # meta = str(i) + '\t' + train_or_test_list[i] + '\t' + labels[i]
        meta = str(i) + '\t' + "train" + '\t' + labels[i]
        meta_data_list.append(meta)
    meta_data_str = '\n'.join(meta_data_list)

    f = open('data/' + dataset_name + '.txt', 'w')
    f.write(meta_data_str)
    f.close()

    corpus_str = '\n'.join(sentences)

    f = open('data/corpus/' + dataset_name + '.txt', 'w')
    f.write(corpus_str)
    f.close()

In [25]:
prepare_data()

In [16]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()
def remove():
    dataset  = 'My_dataset'
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    print(stop_words)
    doc_content_list = []
    f = open('data/corpus/' + dataset + '.txt', 'rb')
    # f = open('data/wiki_long_abstracts_en_text.txt', 'r')
    for line in f.readlines():
        doc_content_list.append(line.strip().decode('latin1'))
    f.close()


    word_freq = {}  # to remove rare words

    for doc_content in doc_content_list:
        temp = clean_str(doc_content)
        words = temp.split()
        for word in words:
            if word in word_freq:
                word_freq[word] += 1
            else:
                word_freq[word] = 1

    clean_docs = []
    for doc_content in doc_content_list:
        temp = clean_str(doc_content)
        words = temp.split()
        doc_words = []
        for word in words:
            if dataset == 'mr':
                doc_words.append(word)
            elif word not in stop_words and word_freq[word] >= 5:
                doc_words.append(word)

        doc_str = ' '.join(doc_words).strip()
        clean_docs.append(doc_str)

    clean_corpus_str = '\n'.join(clean_docs)

    f = open('data/corpus/' + dataset + '.clean.txt', 'w')
    f.write(clean_corpus_str)
    f.close()
    min_len = 10000
    aver_len = 0
    max_len = 0

    f = open('data/corpus/' + dataset + '.clean.txt', 'r')
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        temp = line.split()
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    f.close()
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))

In [26]:
remove()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'who', 'so', 'myself', "don't", 'has', "haven't", 'own', 'those', 'few', 'after', 'he', 'can', 'hasn', 'wouldn', 'their', 'down', 'then', "didn't", "you'd", 'o', 'ourselves', 'shouldn', 'why', 'am', 'your', 'over', 'all', 'than', "weren't", 'it', "mightn't", 'them', "won't", 'where', 'against', 'been', 'other', "should've", 'that', 'yourselves', 'haven', 'some', "needn't", 'whom', 'to', "mustn't", 'further', 'themselves', 'through', 'our', 'aren', 'hadn', "aren't", 'again', 'these', 'until', 'hers', 'how', 've', 'didn', 'weren', 'was', 'at', 'him', 'have', 'do', "you're", 'mustn', 'an', 'from', 'ma', 'for', 'very', 'if', 'more', 'on', 'because', 'any', 'same', 'couldn', 'her', 'as', "you'll", 'in', 'which', "wouldn't", "couldn't", 'is', 'herself', 'this', 'below', 'but', "hadn't", 'having', 'll', 'with', 'ain', 'out', 'm', 'when', 're', 'only', 'his', 'before', "she's", 's', 'of', 'needn', 'nor', 'while', 'or', 'we', 'doing', 'doesn', 'me', 'be', "isn't", "it's", 'd', 'does', 'here', 

In [9]:
def load_glove(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array([float(x) for x in line[1:]])
    return words, word_to_vec_map
words, word_to_vec_map = load_glove("/content/glove.6B.50d.txt")

In [121]:
embed_size = 50
def get_docs():
    docs = []
    label = []
    with open('/content/data/corpus/My_dataset.clean.txt' , "r") as f:
        data = f.readlines()
        for doc in data:
            docs.append(doc.strip())
    with open('/content/data/My_dataset.txt' , 'r') as f:
        lines = f.readlines()
        for l in lines:
            label.append(int(l.strip().split()[-1]))
    return docs , label
def doc_embedding(doc, word_embeddings):
    """
    Tính document embedding bằng cách lấy trung bình các word embedding trong document.

    Args:
        doc (str): Tài liệu (document).
        word_embeddings (dict): Dictionary chứa word embedding tương ứng với mỗi từ.

    Returns:
        dict: Dictionary chứa document embedding, với key là 0 và value là document embedding.
    """
    # Tách từng từ trong document
    words = doc.split()

    # Tính trung bình của các word embedding
    doc_embed = []
    for word in words:
        if word in word_embeddings:
            doc_embed.append(word_embeddings[word])
        else:
            # Nếu từ không có trong word_embeddings, sử dụng vector 0 với cùng kích thước
            doc_embed.append([0] *embed_size)
    try:
        doc_embed = np.sum(doc_embed , axis= 0)/len(doc_embed)
    except:
        import pdb
        pdb.set_trace()
    return doc_embed
def sliding(window_size : int =20):
    docs , label= get_docs()
    vocabs = set()
    sliding_window = []
    for doc in docs :
        # print(doc)
        doc = doc.split()
        vocabs.update(doc)
        if(len(doc) < window_size) :
            sliding_window.append(doc)
        else :
            for i in range(0 , len(doc) - window_size):
                sliding_window.append(doc[i : i + window_size])
    vocabs = list(vocabs)
    size_vocab = len(vocabs)
    node_id = {}
    features = []
    lb= []
    for idx , vocab in enumerate(vocabs):
        node_id[vocab] = idx
        if vocab not in word_to_vec_map:
            word_to_vec_map[vocab] = np.array([0]*embed_size)
        features.append(word_to_vec_map[vocab])

    for idx , doc in enumerate(docs):
        node_id[doc] = idx  + size_vocab + 1
        # print(doc_embedding(doc , word_to_vec_map).shape , "123")
        # break

        if doc=="":
            continue
        features.append(doc_embedding(doc , word_to_vec_map))
        lb.append(label[idx])
    return node_id ,  features ,sliding_window , size_vocab , vocabs , lb



In [122]:
window_size = 20
node_id , features,sliding_window , size_vocab , vocabs , label = sliding(window_size)
print(size_vocab)


1391


In [69]:
from tqdm import tqdm
from collections import defaultdict
def doc_tfidf_embedding(doc, vocab=None):
    # Tạo TfidfVectorizer
    if vocab:
        vectorizer = TfidfVectorizer(vocabulary=vocab)
    else:
        vectorizer = TfidfVectorizer()

    # Tính TF-IDF
    X = vectorizer.fit_transform([doc])

    # Lấy từ vựng
    vocab = vectorizer.get_feature_names_out()

    # Tạo dictionary với từ vựng và vector embedding
    doc_embeds = {word: X[0, i] for i, word in enumerate(vocab)}

    return doc_embeds


def compute_word_stats(docs ):

    Wi = defaultdict(int)
    Wij = defaultdict(int)
    # Lặp qua từng document
    for doc in docs:
        # Tạo set các từ trong document để loại bỏ trùng lặp
        unique_words = set(doc)

        # Cập nhật số lượng documents chứa từng từ
        for word in unique_words:
            Wi[word] += 1

        # Tính toán số lượng documents chứa cả hai từ
        for i in range(len(unique_words)):
            for j in range(i + 1, len(unique_words)):
                word_i = list(unique_words)[i]
                word_j = list(unique_words)[j]
                Wij[(word_i, word_j)] += 1

    return Wi, Wij
def compute_adj(size_vocab , node_id , sliding_window ):
    N = len(list(node_id.keys()))
    adj = np.eye(N , dtype=float)
    docs , label = get_docs()
    for doc in tqdm(docs) :
        word_tfidf = doc_tfidf_embedding(doc , vocab = vocabs)
        for x in word_tfidf:
            try:
                adj[node_id[x]][node_id[doc]] = word_tfidf[x]
                adj[node_id[doc]][node_id[x]] = word_tfidf[x]
            except:
                # continue
                import pdb
                pdb.set_trace()
    # compute PMI
    word_pair = {}
    word_window = {}
    len_sw = len(sliding_window)
    Wi , Wij = compute_word_stats(sliding_window)
    for (word_i, word_j), count in Wij.items():
        adj[node_id[word_i]][node_id[word_j]] = np.log(count*len_sw/(Wi[word_i]*Wi[word_j]))
    return adj


In [31]:
adj = compute_adj(size_vocab , node_id , sliding_window )

100%|██████████| 500/500 [00:23<00:00, 21.64it/s]


In [119]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.5):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.dropout = dropout

    def forward(self, x, adj , size_vocab):
        x = F.relu(self.conv1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.conv2(x, adj)
        out = x[size_vocab :]
        out = F.softmax(out , dim = 0 )
        return out

class GCNConv(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(GCNConv, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.bias = nn.Parameter(torch.FloatTensor(output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        x = x @ self.weight
        x = adj @ x
        x = x + self.bias
        return x

In [ ]:
features = torch.Tensor(np.stack(features))
adj = torch.Tensor(np.array(adj))
print(features.shape)
print(adj.shape)

In [ ]:
num_epochs = 10

model = GCN(50 , 128 , 10  , 0)
opt = torch.optim.AdamW(model.parameters() , lr = 1e-3)
loss_fc = nn.CrossEntropyLoss()
# target = torch.nn.functional.one_hot(torch.tensor(label))
target =  torch.tensor(label)
for epoch in range(num_epochs):
        outputs = model(features , adj, size_vocab )
        # outputs = torch.argmax(outputs , dim = 0)
        loss = loss_fc(outputs, target)
        print(loss)
        opt.zero_grad()
        loss.backward()
        opt.step()